In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import models
import importlib
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt
K = tf.keras.backend

In [19]:
importlib.reload(models)

<module 'models' from '/Users/varun/School/CS182/finalproj/CS182-Spring2020-NLP-Project/models.py'>

In [10]:
df = pd.read_csv("dataset-binary.csv")
df = df.dropna()

In [11]:
train_df = df.iloc[:int(len(df) * .8)]
test_df = df.iloc[int(len(df) * .8):]

In [7]:
def custom_accuracy(y_true, y_pred):
        return K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32)) / 32

def distance(y_true, y_pred):
        total_corr = K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))
        return K.sum(K.cast(K.abs(K.argmax(y_true) - K.argmax(y_pred)), dtype = tf.int32)) / (32 - total_corr)
    
def sum_metrics(y_true, y_pred):
        acc = (32 - K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))) / 32
        total_corr = K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))
        return acc + K.sum(K.cast(K.abs(K.argmax(y_true) - K.argmax(y_pred)), dtype = tf.int32)) / (32 - total_corr)

In [120]:
def sum_metric_on_set(tokenizer, test_reviews, test_stars, model, model_or_func = True):
    test_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_reviews), maxlen = 250, padding = 'post', truncating = 'post')
    pred = None
    if model_or_func:
        pred = model.predict(test_padded)
    else:
        pred = model(test_padded)
    corr = 0
    dist_sum = 0
    for i in range(len(test_stars)):
        if np.argmax(pred[i]) == np.argmax(test_stars[i]):
            corr += 1
        else:
            dist_sum += np.abs(np.argmax(pred[i]) - np.argmax(test_stars[i]))
    print(corr / len(test_stars))
    print(dist_sum / (len(test_stars) - corr))
    print(((dist_sum / (len(test_stars) - corr)) / 4) + 1 - (corr / len(test_stars)))
    print((dist_sum / (len(test_stars) - corr)) + 1 - (corr / len(test_stars)))
    return pred, test_stars

In [26]:
challenge_5 = pd.read_csv("challenge_5.csv")

In [27]:
challenge_6 = pd.read_csv("challenge_6.csv")

In [13]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 5000, oov_token = '<OOV>')
tokenizer.fit_on_texts(train_df["text"])

In [183]:
model = tf.keras.models.load_model("conc.h5", 
                                            custom_objects = {'custom_accuracy' : custom_accuracy, 
                                                             'distance' : distance,
                                                             'sum_metrics' : sum_metrics})

In [188]:
test_stars = []
for i in challenge_5["stars"]:
    temp = [0, 0, 0, 0, 0]
    temp[int(i - 1)] = 1
    test_stars.append(np.array(temp))
test_stars = np.array(test_stars)

In [189]:
pred, true = sum_metric_on_set(tokenizer, challenge_5["text"], test_stars, model)

0.084
1.2882096069868996
1.238052401746725
2.2042096069868995


In [162]:
models.get_most_confused(pred, true, 5)

({'0-1': (0, 0),
  '0-2': (0, 0),
  '0-3': (0, 0),
  '0-4': (0, 0),
  '1-0': (226, 0.6792154084000968),
  '1-2': (61, 0.38608986774428944),
  '1-3': (38, 0.43303837509531723),
  '1-4': (33, 0.5913599792755011),
  '2-0': (0, 0),
  '2-1': (0, 0),
  '2-3': (0, 0),
  '2-4': (0, 0),
  '3-0': (0, 0),
  '3-1': (0, 0),
  '3-2': (0, 0),
  '3-4': (0, 0),
  '4-0': (0, 0),
  '4-1': (0, 0),
  '4-2': (0, 0),
  '4-3': (0, 0)},
 {'0': 0, '1': 500, '2': 0, '3': 0, '4': 0})